In [1]:
'''
Notes on confidence interval calculation:
FIM assumes local linearity near the optimum and yields uncertainty (i.e., standard deviations) around the point estimate, not the true 
mean or median of a parameter distribution. Therefore, the confidence intervals are correctly computed here because the code below gives 
uncertainty ranges centered on those fitted values.
'''

'\nNotes on confidence interval calculation:\nFIM assumes local linearity near the optimum and yields uncertainty (i.e., standard deviations) around the point estimate, not the true \nmean or median of a parameter distribution. Therefore, the confidence intervals are correctly computed here because the code below gives \nuncertainty ranges centered on those fitted values.\n'

In [1]:
# 95% CI from Monte Carlo
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.2e}'.format)

# Data import
file_path = "/Users/elchulito/Library/CloudStorage/OneDrive-polymtlus/0 - A_Database and methodology_PhD/PlasticFADE.xlsx"
sheet_name = "Uncertainty"
data_CI = pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:D", skiprows=1)
data_CI = data_CI.iloc[42:48] # Row index minus 3, change this range for other polymers
print(data_CI)

# Parameter estimates
x_i, tau_i, y_i, theta_i, z_i, eta_i = data_CI.iloc[:, 2].values
# Standard deviations of parameters
x_i_std, tau_i_std, y_i_std, theta_i_std, z_i_std, eta_i_std = data_CI.iloc[:, 3].values

           Process Parameter  Estimate Standard deviation
42  PP degradation       x_i  3.56e-03           3.48e-03
43             NaN     tau_i  1.52e-03           1.06e-01
44             NaN       y_i  2.48e-02           4.98e-04
45             NaN   theta_i  7.61e-01           3.83e-01
46             NaN       z_i  2.71e-05           6.37e-05
47             NaN     eta_i  4.41e-01           1.09e-01


In [3]:
# Input parameters
data_input = pd.read_excel(file_path, sheet_name="Results", usecols="A:D,F", skiprows=13)
data_input = data_input[data_input.iloc[:, 0] == "PP"]  # Change for different polymer types
print(data_input)
data_input.columns = ['Polymer', 'Compartment', 's', 'I_j', 'C_j']

# Monte Carlo setup
N = 10000
np.random.seed(42)
results = []

# Loop through each row of input
for index, row in data_input.iterrows():
    s = row['s']
    I_j = row['I_j']
    C_j = row['C_j']

    # Monte Carlo with log-normal distribution
    N = 10000
    x_i_samples = np.random.lognormal(np.log(x_i), x_i_std, N)
    tau_i_samples = np.random.lognormal(np.log(tau_i), tau_i_std, N)
    y_i_samples = np.random.lognormal(np.log(y_i), y_i_std, N)
    theta_i_samples = np.random.lognormal(np.log(theta_i), theta_i_std, N)
    z_i_samples = np.random.lognormal(np.log(z_i), z_i_std, N)
    eta_i_samples = np.random.lognormal(np.log(eta_i), eta_i_std, N)

    # Compute k_degr for each sample
    k_samples = x_i_samples * (s**tau_i_samples) * (y_i_samples * I_j**theta_i_samples + z_i_samples * C_j**eta_i_samples)
    k_samples = k_samples[np.isfinite(k_samples)]  # Filter invalid samples (good habit, especially when NaNs are found in the CIs)

    # 95% CI in log-space
    log_k = np.log10(k_samples)
    lower_bound = 10 ** np.percentile(log_k, 2.5)
    upper_bound = 10 ** np.percentile(log_k, 97.5)
    k_point = x_i * (s**tau_i) * (y_i * I_j**theta_i + z_i * C_j**eta_i)
    
    results.append({'Compartment': row['Compartment'], 'k_point': k_point, 'CI_lower': lower_bound, 'CI_upper': upper_bound})

# --- Display results ---
results_CI = pd.DataFrame(results)
print("\n", results_CI)
print(f"\n{N - len(k_samples)} out of {N} samples were invalid and removed.")

  Polymer (i) Compartment (j)  SA:V [cm-1]  I_j [W/m2]  C_j [CFU/ml]
0          PP             Air           25    1.00e+01      5.00e-01
1          PP         Topsoil           25    1.00e-01      6.70e+08
2          PP         Subsoil           25    0.00e+00      1.21e+08
3          PP           Beach           25    1.25e+01      1.25e+07
4          PP   Water surface           25    1.00e+01      2.50e+05
5          PP    Water column           25    0.00e+00      3.85e+04
6          PP        Sediment           25    0.00e+00      4.82e+05

      Compartment  k_point  CI_lower  CI_upper
0            Air 5.12e-04  2.02e-04  3.66e-03
1        Topsoil 7.69e-04  1.48e-04  6.61e-03
2        Subsoil 3.55e-04  7.21e-05  2.46e-03
3          Beach 7.37e-04  3.23e-04  5.15e-03
4  Water surface 5.35e-04  2.29e-04  3.84e-03
5   Water column 1.02e-05  4.23e-06  3.16e-05
6       Sediment 3.10e-05  1.01e-05  1.18e-04

0 out of 10000 samples were invalid and removed.


In [5]:
# Write confidence intervals back to Excel (without modifying)
import xlwings as xw

wb = xw.Book(file_path)  # file_path is your existing Excel file path
sheet = wb.sheets["Results"]

start_row = 15  # Change this index for other polymers
sheet.range(f'M{start_row}').options(index=False, header=False).value = results_CI['CI_lower'].values.reshape(-1, 1)
sheet.range(f'N{start_row}').options(index=False, header=False).value = results_CI['CI_upper'].values.reshape(-1, 1)

wb.save()
wb.close()